In [1]:
# EDA Configuration 
# 02_eda_sql.ipynb
from google.cloud import bigquery

PROJECT_ID = "infinite-mantra-480821-v7"
DATASET_ID = "telco_churn_ds"
TABLE_ID   = "customers_native"   # IMPORTANT: native table

bq = bigquery.Client(project=PROJECT_ID)

full_tbl = f"{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}"
full_tbl

'infinite-mantra-480821-v7.telco_churn_ds.customers_native'

In [2]:
# Dataset Size & Sample
# Row count
bq.query(f"""
SELECT COUNT(*) AS n_rows
FROM `{full_tbl}`
""").to_dataframe()

,n_rows
0,7043


In [3]:
# Sample rows
bq.query(f"""
SELECT *
FROM `{full_tbl}`
LIMIT 5
""").to_dataframe()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,9426-SXNHE,Female,0,False,False,2,True,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Month-to-month,False,Bank transfer (automatic),18.75,53.15,False
1,3387-PLKUI,Female,0,True,True,13,True,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Month-to-month,False,Mailed check,18.80,251.25,False
2,3806-YAZOV,Female,0,False,False,3,True,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Month-to-month,False,Mailed check,18.80,56.00,False
3,0620-XEFWH,Male,0,True,True,4,True,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Month-to-month,False,Mailed check,18.85,84.20,False
4,8992-CEUEN,Female,0,False,False,1,True,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Month-to-month,False,Electronic check,18.85,18.85,False


In [4]:
# Target Variable
bq.query(f"""
SELECT
  Churn,
  COUNT(*) AS cnt,
  ROUND(100 * COUNT(*) / SUM(COUNT(*)) OVER (), 2) AS pct
FROM `{full_tbl}`
GROUP BY Churn
ORDER BY cnt DESC
""").to_dataframe()

,Churn,cnt,pct
0,False,5174,73.46
1,True,1869,26.54


In [5]:
#Categorical Distribution
bq.query(f"""
SELECT Contract, COUNT(*) AS cnt
FROM `{full_tbl}`
GROUP BY Contract
ORDER BY cnt DESC
""").to_dataframe()

,Contract,cnt
0,Month-to-month,3875
1,Two year,1695
2,One year,1473


In [6]:
# Internet Service
bq.query(f"""
SELECT InternetService, COUNT(*) AS cnt
FROM `{full_tbl}`
GROUP BY InternetService
ORDER BY cnt DESC
""").to_dataframe()

,InternetService,cnt
0,Fiber optic,3096
1,DSL,2421
2,No,1526


In [7]:
# Payment method
bq.query(f"""
SELECT PaymentMethod, COUNT(*) AS cnt
FROM `{full_tbl}`
GROUP BY PaymentMethod
ORDER BY cnt DESC
""").to_dataframe()

,PaymentMethod,cnt
0,Electronic check,2365
1,Mailed check,1612
2,Bank transfer (automatic),1544
3,Credit card (automatic),1522


In [10]:
# Numerical Deature Summary
bq.query(f"""
SELECT
  AVG(MonthlyCharges) AS avg_monthly,
  MIN(MonthlyCharges) AS min_monthly,
  MAX(MonthlyCharges) AS max_monthly,
  AVG(TotalCharges)   AS avg_total,
  AVG(tenure)         AS avg_tenure,
  MIN(tenure)         AS min_tenure,
  MAX(tenure)         AS max_tenure
FROM `{full_tbl}`
""").to_dataframe()

,avg_monthly,min_monthly,max_monthly,avg_total,avg_tenure,min_tenure,max_tenure
0,64.761692,18.25,118.75,2283.300441,32.371149,0,72


In [12]:
[(f.name, f.field_type) for f in bq.get_table(full_tbl).schema if f.name.lower() == "churn"]

[('Churn', 'BOOLEAN')]

In [13]:
# Tenure by age
bq.query(f"""
WITH base AS (
  SELECT
    Churn,                      -- BOOL
    Contract,
    CASE
      WHEN tenure < 12 THEN '<12'
      WHEN tenure BETWEEN 12 AND 24 THEN '12-24'
      WHEN tenure BETWEEN 25 AND 48 THEN '25-48'
      ELSE '>48'
    END AS tenure_bucket
  FROM `{full_tbl}`
)
SELECT
  Contract,
  tenure_bucket,
  COUNTIF(Churn)                               AS churn_yes,       -- TRUEs
  COUNT(*)                                     AS total,
  ROUND(100 * COUNTIF(Churn) / COUNT(*), 2)    AS churn_rate_pct
FROM base
GROUP BY Contract, tenure_bucket
ORDER BY churn_rate_pct DESC, total DESC
""").to_dataframe()

,Contract,tenure_bucket,churn_yes,total,churn_rate_pct
0,Month-to-month,<12,991,1908,51.94
1,Month-to-month,12-24,311,823,37.79
2,Month-to-month,25-48,264,802,32.92
3,Month-to-month,>48,89,342,26.02
4,One year,>48,82,634,12.93
5,One year,25-48,55,518,10.62
6,One year,12-24,21,219,9.59
7,One year,<12,8,102,7.84
8,Two year,>48,42,1263,3.33
9,Two year,25-48,6,274,2.19
